# Projeto 1 - Ciência dos Dados

Nome: Barbara Martins Damasceno

Nome: Daniel Delattre

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\User\Documents\2 Semestre ENG\Ciência dos Dados\cdadosproj1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
dados = pd.read_excel('MMMercado.xlsx')


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import re 
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@//|]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

In [7]:
#Classificando como categoria
dados.loc[:,'relevância'] = dados['relevância'].astype('category')
#Níveis da categoria
dados.relevância.cat.categories = ['Irrelevante', 'Relevante']


dados.head(5)

,Treinamento,relevância
0,nunca vi um mercado tão cheio,Irrelevante
1,rt @folha: guedes diz que vai acionar stf cont...,Relevante
2,deixando tudo mais barato:\ncupom ifood (app) ...,Irrelevante
3,rt @reporterlacerda: inovação no grenal\n\npre...,Irrelevante
4,@ericat_lol mercado a vista vai ficar parado p...,Relevante


In [8]:
#Dividindo a porcentagem em relevante e irrelevante
base = dados.relevância.value_counts(True, sort=False)
irrelevante = base[0]
relevante = base[1]
print(f"Probabilidade de ser relevante: {relevante}")
print(f"Probabilidade de ser irrelevante: {irrelevante}")


Probabilidade de ser relevante: 0.59
Probabilidade de ser irrelevante: 0.41


Criando o classificar

In [13]:
#Novas folhas de dados criadas a partir da relevancia
dados_relev = dados[dados.relevância == 'Relevante'].copy()
dados_irrelev = dados[dados.relevância == 'Irrelevante'].copy()

import csv
frases_ttrelev = dados_relev.Treinamento.tolist()
clean_relev = []

output=open('relev.txt','w')
for el in frases_ttrelev:
    output.write(el)
    
for i in frases_ttrelev:
    palavra = cleanup(i)
    
print(frases_ttrelev[0:5])
    

#Irrelevante
frases_ttirrelev = dados_irrelev.Treinamento.tolist()
clean_irrelev = []

    
output1=open('irrelev.txt','w')
for el in frases_ttirrelev:
    output1.write(el)
    
for i in frases_ttirrelev:
    palavra = cleanup(i)
    
print(frases_ttirrelev[0:5])


UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f64c' in position 281: character maps to <undefined>

In [11]:
with open("relev.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    relev_raw = arquivo_texto.read()
    
print(relev_raw[0:10])

#Irrelevante
with open("irrelev.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    irrelev_raw = arquivo_texto.read()
    
print(irrelev_raw[0:10])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 53: invalid continuation byte

### CLEANUP 

In [ ]:
relev_c = cleanup(relev_raw.lower())
print(relev_c[:1000])

### SPLITZADA

In [ ]:
relevzada = relev_c.split()
relevzada[:10]

In [ ]:
serie_relev = pd.Series(relevzada)
tabela_relev = serie_relev.value_counts(True)
tabela_relev


### IRRELEVANTE

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**